In [3]:
import pandas as pd
import sqlite3
from sqlalchemy.engine import create_engine
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier
import utils

In [4]:
avito = sqlite3.connect("avito.db")


In [25]:
train = pd.read_sql('SELECT price,item_seq_number,city,category_name,user_type,deal_probability FROM train WHERE price < 10000', avito).dropna()
train

,price,item_seq_number,city,category_name,user_type,deal_probability
0,400.0,2,Екатеринбург,Товары для детей и игрушки,Private,0.12789
1,3000.0,19,Самара,Мебель и интерьер,Private,0.00000
2,4000.0,9,Ростов-на-Дону,Аудио и видео,Private,0.43177
3,2200.0,286,Набережные Челны,Товары для детей и игрушки,Company,0.80323
4,1300.0,9,Чистополь,Товары для детей и игрушки,Private,0.80323
5,500.0,61,Пермь,"Одежда, обувь, аксессуары",Private,0.80323
6,500.0,85,Оренбург,"Одежда, обувь, аксессуары",Private,0.00000
7,400.0,136,Нижний Новгород,Детская одежда и обувь,Company,0.00000
8,1.0,4,Заводоуковск,"Одежда, обувь, аксессуары",Shop,0.00000
9,70.0,15,Оренбург,Товары для детей и игрушки,Private,0.13858


In [26]:
city = preprocessing.LabelEncoder()
city.fit(train['city'])  
list(city.classes_)
train['city'] = city.transform(train['city'])

categ = preprocessing.LabelEncoder()
categ.fit(train['category_name'])  
list(categ.classes_)
train['category_name'] = categ.transform(train['category_name'])

user = preprocessing.LabelEncoder()
user.fit(train['user_type'])  
list(user.classes_)
train['user_type'] = user.transform(train['user_type'])


In [27]:
#train = pd.get_dummies(train[['category_name', 'user_type']])

In [29]:
tpot_x = train[['price','item_seq_number','city','category_name','user_type']].values[:30000]
tpot_y = train['deal_probability'].values[:30000]
tpot_x

array([[4.000e+02, 2.000e+00, 4.140e+02, 4.200e+01, 1.000e+00],
       [3.000e+03, 1.900e+01, 1.174e+03, 2.200e+01, 1.000e+00],
       [4.000e+03, 9.000e+00, 1.153e+03, 2.000e+00, 1.000e+00],
       ...,
       [1.000e+03, 6.100e+01, 1.174e+03, 2.900e+01, 1.000e+00],
       [1.000e+03, 1.000e+00, 5.310e+02, 4.100e+01, 1.000e+00],
       [5.500e+03, 1.510e+02, 3.010e+02, 5.000e+00, 1.000e+00]])

In [30]:
X_train, X_test, y_train, y_test = train_test_split(tpot_x, tpot_y,
                                                    train_size=0.75, test_size=0.25)

In [31]:
#pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
#                                    random_state=42, verbosity=2)

In [34]:
tpot_optimizer = utils.TpotAutoml(mode='regression', max_time_mins=30, scoring='r2',random_state=23, n_jobs=-1,verbosity=2)

In [35]:
tpot_optimizer.fit(X_train, y_train)


Optimization Progress: 100%|██████████| 200/200 [06:37<00:00,  2.85s/pipeline]

Generation 1 - Current best internal CV score: 0.1288542075729566


Optimization Progress: 100%|██████████| 300/300 [11:51<00:00,  2.46s/pipeline]

Generation 2 - Current best internal CV score: 0.12938287198748735


Optimization Progress: 100%|██████████| 400/400 [15:48<00:00,  4.29s/pipeline]

Generation 3 - Current best internal CV score: 0.12938287198748735


Optimization Progress: 100%|██████████| 500/500 [24:14<00:00,  8.16s/pipeline]  

Generation 4 - Current best internal CV score: 0.12938287198748735



32.430257966666666 minutes have elapsed. TPOT will close down.
TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: ExtraTreesRegressor(XGBRegressor(input_matrix, learning_rate=0.01, max_depth=2, min_child_weight=13, n_estimators=100, nthread=1, subsample=0.8), bootstrap=True, max_features=0.7000000000000001, min_samples_leaf=9, min_samples_split=13, n_estimators=100)


In [36]:
top_scores = tpot_optimizer.get_top_models(return_scores=True)
print('\ntop cv scores:')
print(top_scores)
print('\ntop models')
print(tpot_optimizer.top_models)
print('\nthe best test score:')
test_score = tpot_optimizer.score(X_test, y_test)
print(test_score)


top cv scores:
OrderedDict([('ExtraTreesRegressor', 0.12938287198748735), ('XGBRegressor', 0.12764902380254345), ('LassoLarsCV', 0.12683514109289584), ('RidgeCV', 0.12411627868558231), ('RandomForestRegressor', 0.12288545880528338), ('DecisionTreeRegressor', 0.1122301523794286), ('KNeighborsRegressor', 0.05007212134948704), ('GradientBoostingRegressor', 0.04064309850213299), ('LinearSVR', 0.013057045884839491), ('ElasticNetCV', 0.01302112305782488)])

top models
OrderedDict([('ExtraTreesRegressor', {'generation': 'INVALID', 'mutation_count': 2, 'crossover_count': 0, 'predecessor': ('ExtraTreesRegressor(XGBRegressor(input_matrix, XGBRegressor__learning_rate=0.01, XGBRegressor__max_depth=2, XGBRegressor__min_child_weight=13, XGBRegressor__n_estimators=100, XGBRegressor__nthread=1, XGBRegressor__subsample=0.35000000000000003), ExtraTreesRegressor__bootstrap=True, ExtraTreesRegressor__max_features=0.7000000000000001, ExtraTreesRegressor__min_samples_leaf=9, ExtraTreesRegressor__min_sample